In [0]:
# ==============================================================================
# CONFIGURAÇÕES E IMPORTS
# ==============================================================================
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
# Para o XGBoost, a instalação e importação pode variar no Databricks
# Vamos usar o GBTClassifier como um substituto poderoso e nativo do Spark
from pyspark.ml.classification import GBTClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import functions as F
import pandas as pd

# Configurar o uso do nosso catálogo e dos schemas
spark.sql("USE CATALOG previsao_brasileirao")
gold_schema = "gold"
diamond_schema = "diamond"

print(f"Lendo dados de: {gold_schema}")
print(f"Salvando modelos e métricas em: {diamond_schema}")

In [0]:
# ==============================================================================
# PASSO 1: CARREGAR A FEATURE STORE E DIVIDIR EM TREINO/TESTE
# ==============================================================================
print("Carregando feature_store e dividindo os dados...")

# Carregar a tabela final da camada Gold
df_features = spark.table(f"{gold_schema}.feature_store")

# Divisão aleatória dos dados
(df_treino, df_teste) = df_features.randomSplit([0.8, 0.2], seed=42)

print(f"Total de partidas: {df_features.count()}")
print(f"Partidas para Treino (80%): {df_treino.count()}")
print(f"Partidas para Teste (20%): {df_teste.count()}")


# --- AJUSTE DEFINITIVO: INDEXAÇÃO MANUAL VIA JOIN PARA EVITAR O BUG DO .fit() ---

# 1. Obter os labels distintos e atribuir um ID (índice) a cada um.
#    A função window row_number() cria os índices 0, 1, 2...
from pyspark.sql.window import Window
from pyspark.sql import functions as F

df_labels_map = df_features.select("resultado").distinct() \
    .withColumn("label", (F.row_number().over(Window.orderBy("resultado")) - 1).cast("double"))

print("\nDicionário de Mapeamento (Label -> Índice) criado:")
display(df_labels_map)

# 2. Usar um JOIN para adicionar a coluna 'label' aos DataFrames de treino e teste.
#    Esta operação é fundamental no Spark e não sofre do bug de tamanho.
df_treino = df_treino.join(df_labels_map, on="resultado", how="inner")
df_teste = df_teste.join(df_labels_map, on="resultado", how="inner")

# --- FIM DO AJUSTE ---

print("\nAmostra do DataFrame de Treino (agora com a coluna 'label' adicionada via join):")
display(df_treino.limit(5))

In [0]:
# ==============================================================================
# PASSO 2: TREINAR OS MODELOS ESPECIALISTAS (NÍVEL 1)
# ==============================================================================
print("Treinando os 3 modelos especialistas do Nível 1...")

# --- Modelo A: O Generalista (RandomForestClassifier) ---
# Usa todas as features disponíveis
features_gerais_nomes = [col for col in df_treino.columns if col not in ["partida_id", "rodada", "mandante_id", "visitante_id", "resultado", "label"]]

# Converter todas as features para DoubleType para garantir consistência
df_treino_geral = df_treino.select(
    "label",
    *[F.col(c).cast("double").alias(c) for c in features_gerais_nomes]
)

va_geral = VectorAssembler(inputCols=features_gerais_nomes, outputCol="features")

# AJUSTE: Trocado GBTClassifier por RandomForestClassifier, que suporta multiclass
rf_geral = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100, maxDepth=5)

pipeline_geral = Pipeline(stages=[va_geral, rf_geral])
modelo_geral = pipeline_geral.fit(df_treino_geral)
print("  - ✅ Modelo A (Generalista - RandomForest) treinado.")


# --- Modelo B: O Estatístico (Regressão Logística) ---
# Este modelo já suporta multiclass nativamente
features_estatisticas_nomes = ["elo_mandante_pre_jogo", "elo_visitante_pre_jogo", "elo_diff", "mm_gols_m", "mm_gols_v", "diff_mm_gols"]

df_treino_lr = df_treino.select(
    "label",
    *[F.col(c).cast("double").alias(c) for c in features_estatisticas_nomes]
)

va_estatistico = VectorAssembler(inputCols=features_estatisticas_nomes, outputCol="features")
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=20)
pipeline_lr = Pipeline(stages=[va_estatistico, lr])
modelo_lr = pipeline_lr.fit(df_treino_lr)
print("  - ✅ Modelo B (Estatístico - Regressão Logística) treinado.")


# --- Modelo C: O Tático (Random Forest) ---
# Este modelo também já suporta multiclass nativamente
features_taticas_nomes = ["diff_mm_fin", "diff_mm_des"]

df_treino_rf = df_treino.select(
    "label",
    *[F.col(c).cast("double").alias(c) for c in features_taticas_nomes]
)

va_tatico = VectorAssembler(inputCols=features_taticas_nomes, outputCol="features")
rf_tatico = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=50, maxDepth=5)
pipeline_rf = Pipeline(stages=[va_tatico, rf_tatico])
modelo_rf = pipeline_rf.fit(df_treino_rf)
print("  - ✅ Modelo C (Tático - RandomForest) treinado.")

In [0]:
# ==============================================================================
# PASSO 3: GERAR PREVISÕES E AVALIAR CADA ESPECIALISTA INDIVIDUALMENTE
# ==============================================================================
print("Gerando previsões dos especialistas e avaliando cada um...")

# Obter as previsões de cada modelo especialista
pred_geral = modelo_geral.transform(df_teste) 
pred_lr = modelo_lr.transform(df_teste)
pred_rf = modelo_rf.transform(df_teste)

# --- AVALIAÇÃO INDIVIDUAL ---
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

acc_geral = evaluator.evaluate(pred_geral)
acc_lr = evaluator.evaluate(pred_lr)
acc_rf = evaluator.evaluate(pred_rf)

print("\n--- Acurácia dos Modelos Especialistas (Nível 1) ---")
print(f"  - Modelo A (Generalista - RandomForest): {acc_geral:.2%}")
print(f"  - Modelo B (Estatístico - Regressão Log.): {acc_lr:.2%}")
print(f"  - Modelo C (Tático - RandomForest): {acc_rf:.2%}")


# --- PREPARAÇÃO PARA O META-MODELO ---
# Renomear as colunas de probabilidade para evitar conflitos
meta_features_geral = pred_geral.select("partida_id", "label", F.col("probability").alias("prob_geral"))
meta_features_lr = pred_lr.select("partida_id", F.col("probability").alias("prob_lr"))
meta_features_rf = pred_rf.select("partida_id", F.col("probability").alias("prob_rf"))

# Juntar as previsões para formar o DataFrame de treino do Nível 2
df_treino_meta = meta_features_geral.join(meta_features_lr, "partida_id").join(meta_features_rf, "partida_id")

# Criar um único vetor de features para o meta-modelo
assembler_meta = VectorAssembler(inputCols=["prob_geral", "prob_lr", "prob_rf"], outputCol="features")
df_treino_meta = assembler_meta.transform(df_treino_meta)

print("\n✅ Meta-features criadas com sucesso para o Nível 2.")
display(df_treino_meta.limit(5))

In [0]:
# ==============================================================================
# PASSO 4: TREINAR, AVALIAR E SALVAR OS RESULTADOS DO META-MODELO (NÍVEL 2)
# ==============================================================================
print("Treinando e avaliando o meta-modelo final...")

# Usaremos uma Regressão Logística como o "blender" final.
meta_modelo = LogisticRegression(labelCol="label", featuresCol="features", maxIter=20)
modelo_final = meta_modelo.fit(df_treino_meta)

# Fazer as previsões finais no mesmo dataset de teste
previsoes_finais = modelo_final.transform(df_treino_meta)

# Avaliar a performance
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
acuracia = evaluator.evaluate(previsoes_finais)

print(f"\n--- AVALIAÇÃO FINAL DO MODELO HÍBRIDO ---")
print(f"🎯 Acurácia nas últimas 5 rodadas: {acuracia:.2%}")

# Exibir a Matriz de Confusão para uma análise mais detalhada
print("\nMatriz de Confusão:")
display(previsoes_finais.groupBy("label", "prediction").count())


# --- AJUSTE APLICADO AQUI: PERSISTINDO OS RESULTADOS NA CAMADA DIAMOND ---

# Selecionar as colunas mais importantes para a análise de performance
df_resultados_diamond = previsoes_finais.select(
    "partida_id",
    "label",         # O resultado real (0=V_MAND, 1=EMP, 2=V_VIS)
    "prediction",    # A previsão do nosso modelo
    "probability"    # A confiança do modelo em cada uma das 3 classes
)

# Salvar esta tabela de resultados na Camada Diamond
df_resultados_diamond.write \
    .mode("overwrite") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{diamond_schema}.previsoes_validadas")

print(f"\n✅ Tabela de previsões e resultados salva com sucesso em '{diamond_schema}.previsoes_validadas'")

# Em um cenário de produção, você também salvaria o objeto do modelo treinado,
# geralmente usando MLflow, que se integra perfeitamente ao Databricks.
# mlflow.spark.log_model(modelo_final, "modelo_previsao_brasileirao")

In [0]:
# ==============================================================================
# VERIFICAÇÃO PRÉ-EXECUÇÃO: Execute ANTES do Passo 5
# ==============================================================================

print("="*80)
print("VERIFICAÇÃO PRÉ-EXECUÇÃO DO PASSO 5")
print("="*80)

# Lista de variáveis que DEVEM existir
variaveis_necessarias = {
    'modelo_geral': 'Modelo Generalista (RandomForest)',
    'modelo_lr': 'Modelo Estatístico (Logistic Regression)',
    'modelo_rf': 'Modelo Tático (RandomForest)',
    'modelo_final': 'Meta-Modelo (Logistic Regression)',
    'df_labels_map': 'Mapeamento de Labels',
    'acc_geral': 'Acurácia do Modelo Geral',
    'acc_lr': 'Acurácia do Modelo LR',
    'acc_rf': 'Acurácia do Modelo RF',
    'acuracia': 'Acurácia do Meta-Modelo',
    'df_treino': 'DataFrame de Treino',
    'df_teste': 'DataFrame de Teste'
}

print("\n🔍 Verificando variáveis necessárias...\n")

todas_ok = True
variaveis_faltando = []

for var_nome, descricao in variaveis_necessarias.items():
    if var_nome in dir():
        # Verificar o tipo da variável
        var_obj = eval(var_nome)
        tipo = type(var_obj).__name__
        
        # Para DataFrames, mostrar a contagem
        if 'df_' in var_nome:
            try:
                count = var_obj.count()
                print(f"✅ {var_nome:<20} ({descricao}) - {count:,} registros")
            except:
                print(f"✅ {var_nome:<20} ({descricao})")
        # Para métricas, mostrar o valor
        elif 'acc' in var_nome or var_nome == 'acuracia':
            print(f"✅ {var_nome:<20} ({descricao}) - {var_obj:.2%}")
        else:
            print(f"✅ {var_nome:<20} ({descricao})")
    else:
        print(f"❌ {var_nome:<20} ({descricao}) - NÃO ENCONTRADA")
        todas_ok = False
        variaveis_faltando.append(var_nome)

print("\n" + "="*80)

if todas_ok:
    print("✅ TUDO PRONTO! Você pode executar o Passo 5.")
    print("="*80)
else:
    print("❌ ATENÇÃO! Algumas variáveis estão faltando.")
    print("="*80)
    print(f"\nVariáveis faltando: {', '.join(variaveis_faltando)}")
    print("\n🔧 SOLUÇÃO:")
    print("1. NÃO execute o Passo 5 ainda")
    print("2. Volte e execute TODOS os passos anteriores:")
    print("   - PASSO 1: Configurações")
    print("   - PASSO 2: Carregar e dividir dados")
    print("   - PASSO 3: Treinar modelos especialistas")
    print("   - PASSO 4: Gerar previsões e avaliar")
    print("   - (PASSO 4 continuação): Treinar meta-modelo")
    print("3. Aguarde CADA passo terminar completamente")
    print("4. Execute esta verificação novamente")
    print("5. Só execute o Passo 5 quando tudo estiver ✅")
    print("\n" + "="*80)

# Informações adicionais úteis
print("\nℹ️ INFORMAÇÕES DO AMBIENTE:")
print(f"   Catálogo atual: {spark.catalog.currentCatalog()}")
print(f"   Schema atual: {spark.catalog.currentDatabase()}")

# Verificar se o schema diamond existe
try:
    tabelas = spark.sql("SHOW TABLES IN previsao_brasileirao.diamond").collect()
    print(f"   Tabelas em 'diamond': {len(tabelas)}")
except Exception as e:
    print(f"   ⚠️ Schema 'diamond' pode não existir: {e}")

print("\n" + "="*80)

In [0]:
# ==============================================================================
# PASSO 5: SALVAR MODELOS (COM SUPORTE A VOLUMES)
# ==============================================================================
import mlflow
from datetime import datetime
import os

print("="*80)
print("SALVANDO MODELOS - DATABRICKS SERVERLESS")
print("="*80)

schema_destino = "diamond"

# ==============================================================================
# PASSO 1: CRIAR/VERIFICAR VOLUME PARA MODELOS
# ==============================================================================
print("\n[Configuração] Criando estrutura de Volumes...")

# Criar o volume se não existir
try:
    spark.sql(f"""
        CREATE VOLUME IF NOT EXISTS previsao_brasileirao.{schema_destino}.mlflow_models
        COMMENT 'Volume para armazenamento temporário de modelos MLflow'
    """)
    print("  ✅ Volume 'mlflow_models' configurado")
except Exception as e:
    print(f"  ℹ️ Volume já existe ou erro: {e}")

# Definir o caminho do volume
volume_path = f"/Volumes/previsao_brasileirao/{schema_destino}/mlflow_models"
print(f"  📁 Caminho do volume: {volume_path}")

# Configurar variável de ambiente (alternativa 1)
os.environ['MLFLOW_DFS_TMP'] = volume_path

# ==============================================================================
# PASSO 2: CONFIGURAR MLFLOW
# ==============================================================================
mlflow.set_registry_uri("databricks-uc")

# Encerrar run anterior se existir
try:
    mlflow.end_run()
    print("  ℹ️ Run anterior encerrada")
except:
    pass

# Configurar experimento
experiment_name = "/Shared/previsao_brasileirao"
mlflow.set_experiment(experiment_name)
print(f"  📝 Experimento: {experiment_name}")

# ==============================================================================
# PASSO 3: SALVAR MODELOS
# ==============================================================================
print("\n" + "="*80)
print("INICIANDO SALVAMENTO DOS MODELOS")
print("="*80)

with mlflow.start_run(run_name=f"treinamento_{datetime.now().strftime('%Y%m%d_%H%M%S')}") as run:
    
    run_id = run.info.run_id
    print(f"\n🆔 Run ID: {run_id}")
    print(f"📝 COPIE E GUARDE ESTE ID!")
    print("="*80)
    
    # ========== SALVAR MODELO 1: GERAL ==========
    print("\n[1/4] Salvando Modelo Geral...")
    try:
        mlflow.spark.log_model(
            spark_model=modelo_geral,
            artifact_path="modelo_geral",
            dfs_tmpdir=volume_path  # Especificar volume explicitamente
        )
        print("  ✅ Modelo Geral salvo!")
    except Exception as e:
        print(f"  ❌ Erro: {e}")
        raise
    
    # ========== SALVAR MODELO 2: LR ==========
    print("\n[2/4] Salvando Modelo LR...")
    try:
        mlflow.spark.log_model(
            spark_model=modelo_lr,
            artifact_path="modelo_lr",
            dfs_tmpdir=volume_path
        )
        print("  ✅ Modelo LR salvo!")
    except Exception as e:
        print(f"  ❌ Erro: {e}")
        raise
    
    # ========== SALVAR MODELO 3: RF ==========
    print("\n[3/4] Salvando Modelo RF...")
    try:
        mlflow.spark.log_model(
            spark_model=modelo_rf,
            artifact_path="modelo_rf",
            dfs_tmpdir=volume_path
        )
        print("  ✅ Modelo RF salvo!")
    except Exception as e:
        print(f"  ❌ Erro: {e}")
        raise
    
    # ========== SALVAR MODELO 4: FINAL ==========
    print("\n[4/4] Salvando Meta-Modelo...")
    try:
        mlflow.spark.log_model(
            spark_model=modelo_final,
            artifact_path="modelo_final",
            dfs_tmpdir=volume_path
        )
        print("  ✅ Meta-Modelo salvo!")
    except Exception as e:
        print(f"  ❌ Erro: {e}")
        raise
    
    # ========== SALVAR MÉTRICAS ==========
    print("\n" + "-"*80)
    print("Salvando métricas...")
    print("-"*80)
    
    mlflow.log_metric("acuracia_geral", float(acc_geral))
    mlflow.log_metric("acuracia_lr", float(acc_lr))
    mlflow.log_metric("acuracia_rf", float(acc_rf))
    mlflow.log_metric("acuracia_final", float(acuracia))
    
    mlflow.log_param("num_treino", df_treino.count())
    mlflow.log_param("num_teste", df_teste.count())
    mlflow.log_param("data_treinamento", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    mlflow.log_param("volume_path", volume_path)
    
    print("  ✅ Métricas e parâmetros salvos!")
    
    print("\n" + "="*80)
    print("✅ TODOS OS MODELOS SALVOS COM SUCESSO!")
    print("="*80)

# ==============================================================================
# PASSO 4: CRIAR TABELA DE METADADOS
# ==============================================================================
print("\n" + "-"*80)
print("Criando tabela de metadados...")
print("-"*80)

modelos_info = spark.createDataFrame([
    {
        "nome_modelo": "modelo_geral",
        "tipo": "RandomForestClassifier",
        "features": "todas",
        "acuracia": float(acc_geral),
        "run_id": run_id,
        "caminho_mlflow": f"runs:/{run_id}/modelo_geral",
        "volume_path": volume_path,
        "data_registro": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "ativo": True,
        "versao": 1
    },
    {
        "nome_modelo": "modelo_lr",
        "tipo": "LogisticRegression",
        "features": "estatisticas",
        "acuracia": float(acc_lr),
        "run_id": run_id,
        "caminho_mlflow": f"runs:/{run_id}/modelo_lr",
        "volume_path": volume_path,
        "data_registro": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "ativo": True,
        "versao": 1
    },
    {
        "nome_modelo": "modelo_rf",
        "tipo": "RandomForestClassifier",
        "features": "taticas",
        "acuracia": float(acc_rf),
        "run_id": run_id,
        "caminho_mlflow": f"runs:/{run_id}/modelo_rf",
        "volume_path": volume_path,
        "data_registro": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "ativo": True,
        "versao": 1
    },
    {
        "nome_modelo": "modelo_final",
        "tipo": "LogisticRegression (Meta)",
        "features": "meta_features",
        "acuracia": float(acuracia),
        "run_id": run_id,
        "caminho_mlflow": f"runs:/{run_id}/modelo_final",
        "volume_path": volume_path,
        "data_registro": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "ativo": True,
        "versao": 1
    }
])

# Salvar tabela
modelos_info.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable(f"{schema_destino}.modelos_registry")

print(f"  ✅ Tabela '{schema_destino}.modelos_registry' criada!")

# Atualizar label mapping
df_labels_map.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable(f"{schema_destino}.label_mapping")

print(f"  ✅ Tabela '{schema_destino}.label_mapping' atualizada!")

# ==============================================================================
# PASSO 5: VERIFICAÇÃO FINAL
# ==============================================================================
print("\n" + "="*80)
print("VERIFICAÇÃO FINAL")
print("="*80)

print("\n📊 Tabela de Metadados dos Modelos:")
display(spark.table(f"{schema_destino}.modelos_registry"))

print("\n" + "-"*80)
print("INFORMAÇÕES IMPORTANTES")
print("-"*80)
print(f"\n🆔 RUN_ID: {run_id}")
print(f"📁 Volume: {volume_path}")
print(f"\n✅ Para carregar os modelos no notebook de inferência:")
print(f"   import mlflow")
print(f"   import os")
print(f"   os.environ['MLFLOW_DFS_TMP'] = '{volume_path}'")
print(f"   modelo = mlflow.spark.load_model('runs:/{run_id}/modelo_geral', dfs_tmpdir='{volume_path}')")

# ==============================================================================
# PASSO 6: TESTE DE CARREGAMENTO
# ==============================================================================
print("\n" + "-"*80)
print("TESTE: Validando carregamento dos modelos...")
print("-"*80)

try:
    teste = mlflow.spark.load_model(
        f"runs:/{run_id}/modelo_final",
        dfs_tmpdir=volume_path
    )
    print("✅ TESTE PASSOU! Os modelos podem ser carregados.")
    print("   Você pode prosseguir para o notebook de inferência.")
except Exception as e:
    print(f"❌ TESTE FALHOU: {e}")
    print("   ⚠️ Pode haver problemas no notebook de inferência.")

print("\n" + "="*80)
print("✅✅✅ PROCESSO COMPLETO! ✅✅✅")
print("="*80)
print("\n🎯 Próximo passo:")
print("   Execute o notebook '05_Diamond_Inference_Pipeline'")
print("="*80)